Veamos cómo funciona la bolsa de palabras con el texto que usamos para los N-gramas.

In [ ]:
import requests

url = 'https://raw.githubusercontent.com/apimentelaUP/recursos/master/IslaMisteriosa.txt'
page = requests.get(url)
texto = page.text

In [ ]:
import nltk
nltk.download("punkt")

tokens = nltk.word_tokenize(texto,"spanish")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from collections import Counter

BOW = Counter(tokens)

BOW

In [ ]:
BOW.most_common(20)

In [ ]:
nltk.download("stopwords")

palabras_funcionales = nltk.corpus.stopwords.words("spanish")
palabras_funcionales

In [ ]:
tokens_alpha = [i for i in tokens if i.isalpha()]
tokens_filtrados = [i for i in tokens_alpha if i not in palabras_funcionales]

BOW = Counter(tokens_filtrados)
BOW.most_common(20)

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/apimentelaUP/recursos/master/SPAM.csv'
tabla_spam = pd.read_csv(url)

tabla_spam

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(tabla_spam,test_size=0.3)

palabras_funcionales = nltk.corpus.stopwords.words("english")

In [ ]:
BOW_train = []

for texto in train["Message"]:
  tokens=nltk.word_tokenize(texto)
  #tokens = [i for i in tokens if i.isalpha() and i not in palabras_funcionales]
  BOW_train.append(Counter(tokens))

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder

codificador_entrada = DictVectorizer()
codificador_salida = LabelEncoder()

X = codificador_entrada.fit_transform(BOW_train)
y = codificador_salida.fit_transform(train["Category"])

from sklearn.naive_bayes import MultinomialNB

clasificador = MultinomialNB()
clasificador.fit(X,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
BOW_test = []
for texto in test["Message"]:
  tokens = nltk.word_tokenize(texto)
  #tokens = [i for i in tokens if i.isalpha() and i not in palabras_funcionales]
  BOW_test.append(Counter(tokens))

In [ ]:
X_test = codificador_entrada.transform(BOW_test)

y_pred = clasificador.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

y_true = codificador_salida.transform(test["Category"])

print("Exactitud:",accuracy_score(y_true,y_pred))
print("Precision:",precision_score(y_true,y_pred,average="macro"))
print("Exhaustividad:",recall_score(y_true,y_pred,average="macro"))
print("F1 score:",f1_score(y_true,y_pred,average="macro"))

Exactitud: 0.9808612440191388
Precision: 0.981267217630854
Exhaustividad: 0.9412223905955825
F1 score: 0.9599616858237547


In [ ]:
def mas_informativas_para_clase(codificador, clasificador, clase, binario=True, n=10):
    clase_id = list(clasificador.classes_).index(clase)  # se obtiene el índice de la clase que se quiere analizar
    caracteristicas = codificador.get_feature_names()  # la función maneja los nombres originales de las características
    mejores = sorted(zip(clasificador.coef_[clase_id], caracteristicas))[-n:] # Se ordenan, solo queremos las mejores N
    if binario:
      peores = sorted(zip(clasificador.coef_[clase_id], caracteristicas))[:n] # Se ordenan, solo queremos las peores N
      for coeficiente, caracteristica in peores:
        print(1, caracteristica, coeficiente)
      print() 
    for coeficiente, caracteristica in mejores:
        print(clase, caracteristica, coeficiente)

In [ ]:
mas_informativas_para_clase(codificador_entrada,clasificador,0)

1 ''OK -10.071963702508029
1 'Comfort -10.071963702508029
1 'HEART -10.071963702508029
1 'IF -10.071963702508029
1 'Luxury -10.071963702508029
1 'MARRIED -10.071963702508029
1 'Maangalyam -10.071963702508029
1 'Melle -10.071963702508029
1 'Need -10.071963702508029
1 'RE -10.071963702508029

0 you -5.251682136902992
0 or -5.235681795556551
0 your -5.227776616049438
0 call -5.219933438588412
0 & -5.204429252052447
0 a -4.682891972691528
0 , -4.599693028836554
0 ! -4.199845913032613
0 to -4.058248546465227
0 . -3.5736815530315953
